In [2]:
%pip install japanize-matplotlib

Note: you may need to restart the kernel to use updated packages.


In [48]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#例えば、seaborn を利用している場合であれば sns.set() などで描画フォントが seaborn のデフォルトに上書きされ、日本語表示がされなくなります。
#sns.set(font="IPAexGothic") のように利用フォントに IPAexGothic を設定するか、フォント上書き後に japanize_matplotlib.japanize() を利用するなどで日本語表示できるはずです。
import japanize_matplotlib
japanize_matplotlib.japanize()
import pandas as pd
import os
import glob
import re
import shutil

path_to_RawData_folder = '/home/biofunc/data/ラット行動実験データ（生体機能・組織培養室PC）/Data'
def make_graph_from_xlsx(path_to_RawData_folder):
    path_to_graph_folder = path_to_RawData_folder + '/graph'
    if not os.path.exists(path_to_graph_folder):
        os.mkdir(path_to_graph_folder)
    elif os.path.exists(path_to_graph_folder):
        shutil.rmtree(path_to_graph_folder)
        os.mkdir(path_to_graph_folder)
    
    list_of_path_to_xlsx_folder = sorted(glob.glob(path_to_RawData_folder + '/xlsx_from_json' + '*'))

    #print('check point 1')
    #print(list_of_path_to_xlsx_folder)

    for a_path_to_xlsx in list_of_path_to_xlsx_folder:
        # print(a_path)
        list_of_path_to_all_rats_xlsx = sorted(glob.glob(a_path_to_xlsx + '/all_rats' + '*' + '.xlsx'))
        variation_of_this_folder = a_path_to_xlsx.split('.')[1]
        largest_mod_num = 0
        a_path_to_all_rats_xlsx_largest_mod = ''

        #modが最大のxlsxファイルを選択
        for a_path_to_all_rats in list_of_path_to_all_rats_xlsx:
            mod_num = re.findall('mod(\d+)', a_path_to_all_rats)
            if int(mod_num[0]) > largest_mod_num:
                largest_mod_num = int(mod_num[0])
                a_path_to_all_rats_xlsx_largest_mod = a_path_to_all_rats

        # header=[0,1]で、1行目と2行目をヘッダーとして読み込む
        df_of_all_rats = pd.read_excel(a_path_to_all_rats_xlsx_largest_mod, header=[0,1], index_col=[0])

        # make figure
        list_of_data = df_of_all_rats.columns.get_level_values('data').unique().to_list()
        list_of_subject = df_of_all_rats.columns.get_level_values('subject').unique().to_list()
        list_of_type_of_experiment = sorted(set([i.split('_')[0].upper() for i in df_of_all_rats.columns.get_level_values('subject').unique().tolist()]))
        color_map = [plt.cm.tab20(i) for i in range(len(list_of_type_of_experiment) * 2)]
        color_map_index = 0
        fig, axis = plt.subplots(1, len(list_of_data), figsize=(30, 10), dpi=300)
        for a_data_No in range(len(list_of_data)):
            a_data = list_of_data[a_data_No]
            for a_type_No in range(len(list_of_type_of_experiment)):
                a_type = list_of_type_of_experiment[a_type_No].upper()
                
                list_of_this_type_subject = [i for i in list_of_subject if i.split('_')[0].upper() == a_type]
                list_of_this_type_subject.sort()
                for a_subject in list_of_this_type_subject:
                    df_of_this_type_subject = df_of_all_rats[a_data][a_subject]
                    if re.findall('average', a_subject):
                        axis[a_data_No].plot(df_of_this_type_subject, color=color_map[color_map_index], label=a_subject, alpha=1.0)
                    elif not re.findall('std', a_subject) and not re.findall('sem', a_subject):
                        axis[a_data_No].plot(df_of_this_type_subject, color=color_map[color_map_index + 1], label=a_subject, alpha=0.2)
                color_map_index += 2
            color_map_index = 0

        for a_data_No in range(len(list_of_data)):
            a_data = list_of_data[a_data_No]
            colmun = a_data.split('_moving_average')[0]
            axis[a_data_No].set_title(f'{colmun}_{variation_of_this_folder}')
            axis[a_data_No].legend()
            axis[a_data_No].set_xlabel('traial')
            axis[a_data_No].set_ylabel('percentage (%)')
        # テキストをテキストとして出力する設定
        plt.rc("svg", fonttype="none")
        # SVGとして保存
        fig.savefig(f'{path_to_graph_folder}/{variation_of_this_folder}.svg', format='svg')
        # PNGとして保存
        fig.savefig(f'{path_to_graph_folder}/{variation_of_this_folder}.png', format='png')

In [ ]:
make_graph_from_xlsx(path_to_RawData_folder)